# BPE Token Sequence List를 활용한 LSTM+CNN 기반 악성코드 탐지 모델 Test
- accuracy: 0.9912482726853984
- BPE Token을 생성하면서 같이 제작된 BPE Token list를 데이터로 사용했습니다.
- 해당 모델 구조를 학습시키 위해서는 pefile dataset(dikedataset을 추천합니다.)을 utils.py의 함수들을 활용하여 전처리를 한 뒤 사용하셔야 합니다.
- 전처리를 하는 방식에 대해서는 utils 디렉토리에 있는 test.ipynb 파일을 통해 확인하실 수 있습니다.

# 필요 모듈 설치

In [1]:
!pip install pefile
!pip install capstone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 689 kB/s 
  Created wheel for pefile: filename=pefile-2021.9.3-py3-none-any.whl size=68844 sha256=60cbba9938bf3c14a7685924b51b116f3e44a11c4a7d8c4dc9f70f3a33f67cb1
  Stored in directory: /root/.cache/pip/wheels/50/be/9b/d19a6151b9c6e303f823faedf03193d7e90dac4009a0bdafeb
Successfully built pefile
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 13.1 MB/s 


# utils.py 파일이 있는 디렉토리로 위치 이동

In [5]:
%cd /content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/휴지통/test

/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/휴지통/test


# 필요 패키지 로드

In [6]:
from utils import *

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.layers import Conv1D, MaxPooling1D, Dropout, Activation
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.utils.np_utils import to_categorical

import tensorflow as tf

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# Data load

# opcode list load

In [18]:
filePath = 'opcodesList.txt'
with open(filePath, 'rb') as lf:
    opcodes = pickle.load(lf)

opcodes2 = []
for i in range(len(opcodes)):
    opcodes2.append([opcodes[i]])

print(opcodes[:10])

['aaa', 'aad', 'aam', 'aas', 'adc', 'add', 'and', 'call', 'cbw', 'clc']


## vocab load

In [19]:
with open('2000vocab.p', 'rb') as file:
    vocab = pickle.load(file)

vocab = vocab + opcodes2

print(vocab[:10])

[['add', 'add'], ['mov', 'mov'], ['add', 'add', 'add', 'add'], ['push', 'push'], ['add', 'add', 'add', 'add', 'add', 'add', 'add', 'add'], ['push', 'call'], ['push', 'mov'], ['dec', 'mov'], ['mov', 'mov', 'mov', 'mov'], ['pop', 'pop']]


# 학습된 모델의 구조 생성

In [12]:
model= Sequential()
model.add(Embedding(len(vocab), 3000))
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding = 'valid', activation = 'relu', strides = 1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55))
model.add(Dense(48, activation='relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 3000)        6336000   
                                                                 
 dropout (Dropout)           (None, None, 3000)        0         
                                                                 
 conv1d (Conv1D)             (None, None, 64)          960064    
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 64)         0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 55)                26400     
                                                                 
 dense (Dense)               (None, 48)                2688      
                                                        

# 모델 가중치 Load

In [13]:
# 모델을 저장할 경로를 설정해주시면 됩니다.
filename = 'BPESequenceBasedLSTMCNNMalwareDetectionModel.h5'

model.load_weights(filename)

# Sample file Test
- Test를 위해 utils 폴더 내부의 utils.py 파일을 활용한다.

## MalwareDetectionFunction을 통한 Malware 탐지
- 양성 샘플 파일이 양성으로 결과가 나오는지 확인
- return 1: 악성코드, return 0: 양성코드

In [14]:
# 머신러닝 모델(XGBClassifier)을 바탕으로 해당 파일이 악성코드인지를 알려주는 함수
# return 1: 악성코드, return 0: 양성코드
def MalwareDetectionFunction(model, fileName):
    try:
        #parse exe file
        exe = pefile.PE(fileName)
        #call the function we created earlier
        sampleOpcodeList = ExtractPefileOpcodes(opcodes, exe)
    except:
        print('Error File')
    
    sampleTokenList = Tokenizer(vocab, sampleOpcodeList)
    sampleTokenIdList = TokenIdMapping(vocab, sampleTokenList)

    sampleTokenIdList = np.array([sampleTokenIdList])
    sampleTokenIdList = sequence.pad_sequences(sampleTokenIdList, maxlen=3000)

    y_pred = model.predict(sampleTokenIdList)
    Y_pred = np.argmax(y_pred,axis=1)

    return Y_pred[0]

- 결과가 0으로 양성파일을 탐지한 것을 확인할 수 있다.

In [15]:
fileName = '3f3fe9ecad7f30fc80cdfb678d7ca27a30d0575a73818746e98be9170d3be348.exe'

MalwareDetectionFunction(model, fileName)

0

# opcode Sequence List pickle을 입력으로 사용하는 탐지 함수
- 악성코드 파일을 직접 Google Drive 및 GitHub에 올릴 수 없기 때문에 pickle로 먼저 opcodeSequence를 추출하여 해당 파일을 바탕으로 악성코드를 탐지하는 함수를 제작하였다.

In [16]:
# 머신러닝 모델(XGBClassifier)을 바탕으로 해당 파일이 악성코드인지를 알려주는 함수
# return 1: 악성코드, return 0: 양성코드
def MalwareDetectionFunctionUsingPickle(model, pickleName):
    with open(pickleName, 'rb') as lf:
        sampleOpcodeList = pickle.load(lf)

    sampleTokenList = Tokenizer(vocab, sampleOpcodeList)
    sampleTokenIdList = TokenIdMapping(vocab, sampleTokenList)

    sampleTokenIdList = np.array([sampleTokenIdList])
    sampleTokenIdList = sequence.pad_sequences(sampleTokenIdList, maxlen=3000)

    y_pred = model.predict(sampleTokenIdList)
    Y_pred = np.argmax(y_pred,axis=1)

    return Y_pred[0]

- 악성코드 파일이기 때문에 모델이 1로 분류하는 것을 확인할 수 있다.

In [17]:
pickleName = 'MalwareSample.p'

MalwareDetectionFunctionUsingPickle(model, pickleName)

1